# CLIMADA - PIK data flow: EXAMPLE 1

In this notebook we show one simple impact calculations with CLIMADA using a datafile for floods from the PIK model.

In [1]:
from pathlib import Path

**Define the path to DATA folder. The DATA can be downloaded from NEXTCLOUD [https://cloud.tu-braunschweig.de/f/569399210](https://cloud.tu-braunschweig.de/f/569399210)**   

In [2]:
data_path = Path('/Users/vwattin/Documents/Programming/Climate/DIRECTED/Coupling/DATA')

# Define the output folder path (used later in the script)
output_path = Path('Results')

# Create the output folder if it doesn't exist
output_path.mkdir(parents=True, exist_ok=True)

First, we load the hazard dataset (which contains flood depths for each day between 1971 and 2023 for a total of 19358 events) from the PIK.

In [3]:
haz_folder = 'PIK_Model'
haz_path = data_path / haz_folder
haz_file = 'flddph_7123.nc4'
hazard_type = 'RF' # Type of the hazard, ideally a two-letter identifier. Is needed to match with the exposures and impact functions.
intensity_unit = 'm' # Just for plots
intensity='water_depth' # Which variable in the dataset is to be considered the intensity? 
coordinate_vars = {'longitude': 'lon', 'latitude': 'lat'} # CLIMADA uses latitude and longitude as default x/y coordinate names. This file contains lat/lon. 
crs = 'EPSG:4326' # Assumed, however it is unclear from the dataset whether this is correct. That would need to be included in the netcdf file.

# Read the hazard data
from climada.hazard import Hazard
haz = Hazard.from_xarray_raster_file(
    filepath=haz_path/haz_file,
    hazard_type=hazard_type,
    intensity_unit=intensity_unit,
    intensity=intensity,
    coordinate_vars=coordinate_vars,
    crs=crs
)

NOTE: Some processing might be needed to transform the ouput from the PIK model into a directly usable input for CLIMADA. Here we use the data directly, which works technically, but does not necessarilly produce scientifically sound outputs.

In [4]:
# The frequency of all events must adjusted to the value of interest. Typically, one is interested in annual probabilities. Ideally, this would be included in the netcdf file.
import datetime
import numpy as np
first_year = datetime.datetime.fromordinal(np.min(haz.date)).year
last_year = datetime.datetime.fromordinal(np.max(haz.date)).year
haz.frequency = np.repeat(1/(last_year - first_year), haz.size)

Second, we load an exposure file from the CLIMADA API.

In [5]:
# Add to each location of the hazard the country iso code. This can then be used to get the correct exposure file. 
haz.centroids.set_region_id()

In [6]:
regions = np.unique(haz.centroids.region_id)
regions = regions[regions>0] # 0 is the id for points on the ocean (so do not belong to a land-mass of a country) and should be removed from the list of countries to retrieve

In [7]:
# Get the assset exposures of the LitPop model from the CLIMADA data API
from climada.util.api_client import Client
import climada.util.coordinates as u_coord
client = Client()
all_regions_exposures =  []
for reg in u_coord.country_to_iso(countries=regions, representation='alpha3'): # the method requires iso3alpha codes
    try:
        all_regions_exposures.append(client.get_litpop(reg))
    except LookupError:
        print(f"Country {reg} not found")

Country XKO not found


In [8]:
from climada.entity import Exposures
exp = Exposures.concat(all_regions_exposures) # Make one single exposure file for compute the values for all countries at once.

Third, we define an impact function. This is tricky, as this typically depends strongly on the datasets, and the user should define them carefully. Here we simply assume a 1m threshold for certain countries, and 2m for others (just for demonstration)

In [9]:
from climada.entity import ImpactFunc, ImpactFuncSet
min_depth = 0
max_depth = 1000

threshold1 = 1
impf_id1 = 1
impf1 = ImpactFunc.from_step_impf(
    intensity=(min_depth, threshold1, max_depth),
    haz_type=hazard_type,
    impf_id=impf_id1
)

threshold2 = 2
impf_id2 = 2
impf2 = ImpactFunc.from_step_impf(
    intensity=(min_depth, threshold2, max_depth),
    haz_type=hazard_type,
    impf_id=impf_id2
)

impfset = ImpactFuncSet([impf1, impf2])

Fourth, we need to assign the impact functions to the exposure points. Here we choose certain countries at random to have 1m, others 2m.

In [10]:
exp.gdf[f'impf_{hazard_type}'] = impf_id1
exp.gdf[exp.gdf['region_id'] > 150 ][f'impf_{hazard_type}'] = impf_id2

In [11]:
# We can assign centroids of the hazard the exposure points manually. This is useful when doing repeated impact calculations
exp.assign_centroids(haz, threshold=10) #threshold for nearest neighbour assignement is chosen to 10km (arbitrary choice, default would be 100)

2024-08-28 15:24:27,760 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 10km for 171360 coordinates.


Fith, we compute the impacts

In [12]:
from climada.engine import ImpactCalc
impcalc = ImpactCalc(exposures=exp, impfset=impfset, hazard=haz)

In [13]:
impact = impcalc.impact(assign_centroids=False)

Sixth, we can extract diverse risk and impact metrics. One can also save them to file.

In [14]:
# Average annual impact over the whole region
impact.aai_agg

270380992296766.94

In [15]:
# Average annual impact per exposure point
impact.eai_exp

array([0., 0., 0., ..., 0., 0., 0.])

In [1]:
# Total impact per event
impact.at_event

NameError: name 'impact' is not defined

In [17]:
# Impact at given return periods
impact.calc_freq_curve(return_per=[1,2,5,10,20])

ImpactFreqCurve(return_per=[1, 2, 5, 10, 20], impact=array([1.56725719e+12, 1.68103937e+12, 1.79433346e+12, 1.96842222e+12,
       2.02652445e+12]), unit='USD', frequency_unit='1/year', label='Exceedance frequency curve')

In [18]:
# Save to csv (netcdf will be implemented soon)
impact.write_csv(output_path / 'impact_pik_example1.csv')

In [19]:
# The full event/point impact matrix
impact.imp_mat

<19358x264734 sparse matrix of type '<class 'numpy.float64'>'
	with 95950630 stored elements in Compressed Sparse Row format>

In [20]:
# Save the full event/point impact matrix in sparse format
impact.write_sparse_csr(output_path / 'impact_pik_example1_impmat.npz')